### Librerías

In [1]:
import csv
import pandas as pd
import numpy as np

### Obtener datos

In [2]:
%store -r estudiantes_d
%store -r vig_info_d
%store -r novig_info_d

estudiantes = estudiantes_d
vig_info = vig_info_d
novig_info = novig_info_d

### Crear log de eventos

1er semestre:
* 01/01 Situación semestral (Suspensión, PPA, PPS, etc)
* 10/01 Alerta
* 20/01 Revisión Permanencia
* 31/01 Situación Final

2do semestre:
* 01/08 Situación semestral
* 10/08 Alerta
* 20/08 Revisión Permanencia
* 31/08 Situación Final

In [3]:
def creador_timestamp(periodo_int, tipo):
    periodo = str(periodo_int)
    timestamp = 0
    if periodo[-1] == '0': # Primer semestre
        if tipo == 'situacion':
            timestamp = '01-'+periodo[0:4]
        elif tipo == 'alerta':
            timestamp = '01-'+periodo[0:4]
        elif tipo == 'revision':
            timestamp = '02-'+periodo[0:4]
        elif tipo == 'fin':
            timestamp = '07-'+periodo[0:4]
    elif periodo[-1] == '2': # Segundo semestre
        if tipo == 'situacion':
            timestamp = '08-'+periodo[0:4]
        elif tipo == 'alerta':
            timestamp = '08-'+periodo[0:4]
        elif tipo == 'revision':
            timestamp = '11-'+periodo[0:4]
        elif tipo == 'fin':
            timestamp = '12-'+periodo[0:4]
    return timestamp

In [4]:
sit_finales = {
    'INPROGRESS - En progreso': 'Taking courses',
    'ANULESTUDIOS - Anulación de estudios': 'Taking courses',
    'SUSPESTUDIOS - Suspensión de estudios': 'Taking courses',
    'SUSPESTCOMPERMA - Susp. Estudios Com.Permanencia': 'Taking courses',
    'SUSPSALUD - Suspensión por salud': 'Taking courses',
    'SUSPFUERZMAYOR - Suspensión por fuerza mayor': 'Taking courses',
    'SUSPSANCACADEM - Suspensión sanción académ.': 'Taking courses',
    'RENUNCIADO - Renunciado': 'Dropout',
    'RENUNCIACOMPERM - Renuncia Com. Permanencia': 'Dropout',
    'ABANDONA - Abandono de estudios': 'Dropout',
    'ELIMINADO - Eliminado': 'Expelled',
    'NOVIGCOMPERMA - No Vigente Com. Permanencia': 'Expelled',
    'EGRESADO - Egresado': 'Graduated',
    'VIGENTEGRESADO - Vigente egresado': 'Graduated',
    'AVT - Alumno en Vía de Titulación': 'Graduated'
}

In [5]:
sit_sem_activa = [
    'INPROGRESS - En progreso',
    'IACAD - Intercambio Académico',
    'INPROGRESS - En progreso (Reintegro)', 
]

sit_sem_susp = [
    'SUSPESTUDIOS - Suspensión de estudios',
    'SUSPESTUDIOS - Suspensión de estudios (Reintegro)',
    'SUSPSALUD - Suspensión por salud',
    'SUSPSALUD - Suspensión por salud (Reintegro)',
    'SUSPSANCACADEM - Suspensión sanción académ.',
    'SUSPFUERZMAYOR - Suspensión por fuerza mayor',
    'ANULESTUDIOS - Anulación de estudios'
]

In [6]:
reales = [201920, 201922, 202020, 202022, 202120, 202122]
def es_real(periodo):
    if periodo in reales:
        return True
    else:
        return False

In [7]:
def agregar_info(evento, info):
    evento.append(info[rut]['PERIODO ADMISION'])
    evento.append(info[rut]['TIPO ADMISION'])
    evento.append(info[rut]['SEXO'])
    evento.append(info[rut]['VIGENCIA'])
    evento.append(info[rut]['REGION'])
    evento.append(info[rut]['ESTABLECIMIENTO EDUCACIONAL'])
    evento.append(info[rut]['PTJ NEM'])
    evento.append(info[rut]['PTJ RANKING'])
    evento.append(info[rut]['PTJ SELECCION'])
    evento.append(info[rut]['PREFERENCIA'])
    evento.append(info[rut]['PUESTO INGRESO'])
    evento.append(info[rut]['ESTADO CIVIL'])
    evento.append(info[rut]['VIVE CON'])
    evento.append(info[rut]['SISTEMA SALUD'])

In [8]:
log = [['ID',
        'Activity',
        'Timestamp',
        'CGPA',
        'SGPA',
        'SGPA range',
        'Num. sem PPS < 4.0',
        'Periodo admisión',
        'Tipo admisión',
        'Sexo',
        'Vigencia',
        'Región',
        'Establecimiento educacional',
        'Ptj NEM',
        'Ptj Ranking',
        'Ptj Selección',
        'Preferencia',
        'Puesto ingreso',
        'Estado Civil',
        'Vive con',
        'Sistema de Salud'
]]

situ = []

for rut in estudiantes.keys():
    num_alerta = 1
    PPS_4 = 0
    # Información del estudiante
    if rut in vig_info.keys():
            info = vig_info
    elif rut in novig_info.keys():
        info = novig_info

        
    for semestre in range(1,30):
        if semestre in estudiantes[rut].keys():
            if estudiantes[rut][semestre]['PPS'] > 0:              
                
                # Alerta / Sin Alerta
                evento_alerta = []
                evento_alerta.append(rut) # Rut            
                if 'ALERTA' in estudiantes[rut][semestre].keys(): 
                    PPS_4 = 0
                    evento_alerta.append('Alert') # Actividad
                    evento_alerta.append(creador_timestamp(estudiantes[rut][semestre]['PERIODO ACADEMICO'], 'alerta')) # Timestamp
                    evento_alerta.append(estudiantes[rut][semestre]['PPA'])
                    evento_alerta.append(estudiantes[rut][semestre]['PPS'])
                    if estudiantes[rut][semestre]['PPS'] < 3.0:
                        evento_alerta.append('SGPA < 3.0')
                    elif estudiantes[rut][semestre]['PPS'] < 3.5:
                        evento_alerta.append('3.0 <= SGPA < 3.5')
                    elif estudiantes[rut][semestre]['PPS'] < 4.0:
                        evento_alerta.append('3.5 <= SGPA < 4.0')
                    else:
                        evento_alerta.append('SGPA >= 4.0')
                    evento_alerta.append('') # Num sem PPS < 4.0
                
                else:
                    if estudiantes[rut][semestre]['PPS'] < 4.0:
                        PPS_4 += 1
                    evento_alerta.append('No Alert') # Actividad
                    evento_alerta.append(creador_timestamp(estudiantes[rut][semestre]['PERIODO ACADEMICO'], 'alerta')) # Timestamp
                    evento_alerta.append(estudiantes[rut][semestre]['PPA'])
                    evento_alerta.append(estudiantes[rut][semestre]['PPS'])
                    if estudiantes[rut][semestre]['PPS'] < 3.0:
                        evento_alerta.append('SGPA < 3.0')
                    elif estudiantes[rut][semestre]['PPS'] < 3.5:
                        evento_alerta.append('3.0 <= SGPA < 3.5')
                    elif estudiantes[rut][semestre]['PPS'] < 4.0:
                        evento_alerta.append('3.5 <= SGPA < 4.0')
                    else:
                        evento_alerta.append('SGPA >= 4.0')
                    evento_alerta.append(f'sem. PPS < 4.0 = {PPS_4}')
                agregar_info(evento_alerta, info)
                log.append(evento_alerta)
              

            # Suspensión
            elif estudiantes[rut][semestre]['SITUACION ACADEMICA'] in sit_sem_susp:
                evento_susp = []
                evento_susp.append(rut) # Rut
                evento_susp.append('Stopout') # Actividad
                evento_susp.append(creador_timestamp(estudiantes[rut][semestre]['PERIODO ACADEMICO'], 'alerta')) # Timestamp
                evento_susp.append('') # PPA
                evento_susp.append('') # PPS
                evento_susp.append('') # PPS rango
                evento_susp.append('') # Num sem PPS < 4.0
                agregar_info(evento_susp, info)
                log.append(evento_susp)
                
            # Otra situación
            elif estudiantes[rut][semestre]['SITUACION ACADEMICA'] in sit_sem_activa:
                evento_otro = []
                evento_otro.append(rut) # Rut
                evento_otro.append('Otra situación académica') # Actividad
                evento_otro.append(creador_timestamp(estudiantes[rut][semestre]['PERIODO ACADEMICO'], 'situacion')) # Timestamp
                evento_otro.append('') # PPA
                evento_otro.append('') # PPS
                evento_otro.append('') # PPS rango
                evento_otro.append('') # Num sem PPS < 4.0
                agregar_info(evento_otro, info)
                log.append(evento_otro)

    # Situación final
    evento_sit_final = []
    evento_sit_final.append(rut)
    evento_sit_final.append(sit_finales[info[rut]['SITUACION FINAL']])
    ultimo_sem = estudiantes[rut][max(estudiantes[rut].keys())]['PERIODO ACADEMICO']
    evento_sit_final.append(creador_timestamp(ultimo_sem, 'fin'))
    evento_sit_final.append('') # PPA
    evento_sit_final.append('') # PPS
    evento_sit_final.append('') # PPS rango
    evento_sit_final.append('') # Num sem PPS < 4.0
    agregar_info(evento_sit_final, info)
    log.append(evento_sit_final)

### Crear excel

In [9]:
myFile = open('../../03 Logs/02 Dataset_100/model3_law.csv', 'w', newline="")
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(log)